### Explainer Experiment Demo

Below we show some examples on how you can use the **Experiment_Explainers** class. The general steps consist of:
1. prepare data
2. Initialize recommenders, explainers and metrics
3. Initialize experiment and run the experiment

You can set distribution=True to enable distribution plots while running the experiments. The plots will be saved in /experiment_plots.

When the experiment is completed, result table is saved in /experiment_plots.

In [2]:
import warnings
from cornac.datasets import goodreads
from cornac.experiment.experiment_explainers import Experiment_Explainers
from cornac.models import EMF, ALS, FMRec, MTER, EFM, NEMF
from cornac.explainer import (
    Exp_EFM,
    Exp_LIMERS,
    Exp_MTER,
    Exp_ALS,
    Exp_PHI4MF,
    Exp_SU4EMF,
)
from cornac.datasets.goodreads import prepare_data
from cornac.metrics_explainer import (
    Metric_Exp_PSPNFNS as PSPN,
    Metric_Exp_PGF as PGF,
    Metric_Exp_MEP as MEP,
    Metric_Exp_EnDCG as EnDCG,
)
from cornac.metrics_explainer import (
    Metric_Exp_DIV as DIV,
    Metric_Exp_FPR as FPR,
    Metric_Exp_FA as FA,
    Metric_Exp_RA as RA,
)

warnings.filterwarnings("ignore")

/Users/yaoyao/miniconda3/envs/rose/lib/python3.9/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


**Experiment 1**: run PSPNFNS, FDIV and FPR on (FM, Exp_LIMERS) pair using rating data and sentiment data from amazon toys. Since this dataset does not contain explicit item and user features, we parsed aspect and opinion from sentiment data as item and user features respectively in the  ***create_item_features_from_aspects*** function.

In [5]:
from cornac.datasets import amazon_toy
import numpy as np
from cornac.data import FeatureModality
from cornac.eval_methods import RatioSplit


def create_item_features_from_aspects(at_sentiment):
    """Separate aspects and opinions from sentiment data and create item and user features from them."""
    items = {}
    users = {}
    for _, row in enumerate(at_sentiment):
        user, item, sentiments = row
        if user not in users:
            users[user] = []
        if item not in items:
            items[item] = []
        for sentiment in sentiments:
            if sentiment[0] not in items[item]:
                items[item].append(sentiment[0])  # aspect adds to item feature
            if sentiment[1] not in users[user]:
                users[user].append(sentiment[1])  # opinion adds to user feature

    item_aspect_pairs = np.array(
        [(item, feature) for item in items for feature in items[item]]
    )
    user_opinion_pairs = np.array(
        [(user, feature) for user in users for feature in users[user]]
    )
    return item_aspect_pairs, user_opinion_pairs, items.keys(), users.keys()


at_feedback = amazon_toy.load_feedback()
at_feedback = at_feedback[: len(at_feedback) // 20]  # reduce data size
at_sentiment = amazon_toy.load_sentiment()
items_feature, users_feature, items_list, users_list = (
    create_item_features_from_aspects(at_sentiment)
)
# remove unknown users and items from rating data
at_feedback_excl_unknowns = [
    x for x in at_feedback if x[0] in users_list and x[1] in items_list
]

In [6]:
# prepare data
rs = RatioSplit(
    data=at_feedback,
    test_size=0.2,
    item_feature=FeatureModality(items_feature),
    # user_feature=FeatureModality(users_feature), # user feature is not used in this experiment
    seed=42,
    exclude_unknowns=True,
)
# initialize recommenders, explainers and metrics
fm = FMRec()
limers = Exp_LIMERS(rec_model=fm, dataset=rs.train_set)
pspnfns = PSPN()
fdiv = DIV()
fpr = FPR()

# initialize experiment
models = [(fm, limers)]
metrics = [pspnfns, fdiv, fpr]
experiment = Experiment_Explainers(
    eval_method=rs,
    models=models,
    metrics=metrics,
    distribution=True,
    rec_k=4,
    feature_k=4,
    eval_train=True,
)
experiment.run()

INFO:cornac.experiment.experiment_explainers:Start training Recommender FMRec...


Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training MSE: 0.84335
-- Epoch 2
Training MSE: 0.54054
-- Epoch 3
Training MSE: 0.47963
-- Epoch 4
Training MSE: 0.43793
-- Epoch 5
Training MSE: 0.40798
-- Epoch 6
Training MSE: 0.38155
-- Epoch 7
Training MSE: 0.36740
-- Epoch 8
Training MSE: 0.35302
-- Epoch 9
Training MSE: 0.34088
-- Epoch 10


INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'FMRec:Exp_LIMERS'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender FMRec creates recommendations


Training MSE: 0.33139


INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_LIMERS create explanation for all recommendations
Computing explanations: : 20537it [07:39, 44.66it/s]                         
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_PSPNFNS starts evaluation...
INFO:cornac.experiment.experiment_explainers:self.current_rec: FMRec, self.current_exp: Exp_LIMERS
Re-evaluate after features removal... : 20537it [06:49, 50.21it/s]                           
INFO:cornac.experiment.experiment_explainers:Result: Probability of Necessity: 0.9873071979434447; Probability of Sufficiency: 0.0; Harmonic Mean: 0.0
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_DIV starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Feature diversity: 0.08820979829202373
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_FPR starts evaluation...
ERROR:cornac.experiment.experiment_explainers:Metric Metric_Exp_FPR d

**Experiment 2**: run PSPN, FDIV and FPR on (EFM, Exp_EFM) and (MTER, Exp_MTER) pairs using data from goodreads. This dataset contains explicit review keywords.

In [3]:
import pandas as pd
from cornac.data import Reader
from cornac.datasets import amazon_digital_music
from cornac.data.lexicon import SentimentAnalysis
from cornac.eval_methods import RatioSplit
from cornac.data import SentimentModality

feedback = amazon_digital_music.load_feedback()
review = amazon_digital_music.load_review()
columns = ["user_id", "item_id", "rating", "review_text"]
ratings_df = pd.DataFrame(feedback, columns=columns[:3])
reviews_df = pd.DataFrame(review, columns=[columns[0], columns[1], columns[3]])
all_df = pd.merge(ratings_df, reviews_df, on=["user_id", "item_id"])
input_df = all_df[: 1000] # reduce data size
SA = SentimentAnalysis(input_df, usecols = columns) 
df = SA.build_lexicons()
output_lexicon = './dataset/lexicon.txt'
output_rating = './dataset/rating.txt'
SA.save_to_file(output_lexicon, output_rating)

100%|██████████| 1000/1000 [00:37<00:00, 26.93it/s]

number of users: 682
number of items: 54
61 rows have no lexicon
939 rows after dropping users having less than 1 reviews


In [4]:
reader = Reader()
ratings = reader.read(output_rating, fmt='UIR', sep=',')
lexicon = reader.read(output_lexicon, fmt='UITup', sep=',', tup_sep=':')
sentiment = SentimentModality(data = lexicon)

In [5]:
# prepare data
rs = RatioSplit(
    data=ratings,
    test_size=0.2,
    sentiment=sentiment,
    seed=42
)
ground_truth_good_reads = lexicon

# initialize recommenders and explainers
efm = EFM(
    max_iter=20,
    num_explicit_factors=50,
    num_latent_factors=50,
    num_most_cared_aspects=50,
    alpha=0.85,
    lambda_h=0.1,
    lambda_u=0.001,
    lambda_v=0.01,
    lambda_x=1.0,
    lambda_y=0.1,
)
efm_exp = Exp_EFM(rec_model=efm, dataset=rs.train_set)
mter = MTER(
    max_iter=20,
    n_aspect_factors=8,
    n_item_factors=5,
    n_opinion_factors=5,
    n_user_factors=10,
    lambda_bpr=10,
    lambda_reg=10,
    n_bpr_samples=1000,
    n_element_samples=50,
)
mter_exp = Exp_MTER(rec_model=mter, dataset=rs.train_set)

# initialize metrics
pspnfns = PSPN()
fdiv = DIV()
fpr = FPR()
fpr_with_input_as_groundtruth = FPR(ground_truth=ground_truth_good_reads)

# initialize experiment
models = [(efm, efm_exp), (mter, mter_exp)]
metrics = [fdiv, fpr_with_input_as_groundtruth, pspnfns]
experiment = Experiment_Explainers(
    eval_method=rs,
    models=models,
    metrics=metrics,
    rec_k=10,
    distribution=False,
    feature_k=10,
    eval_train=True,
)
experiment.run()

INFO:cornac.experiment.experiment_explainers:Start training Recommender EFM...


INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'EFM:Exp_EFM'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender EFM creates recommendations
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_EFM create explanation for all recommendations
Computing explanations: 100%|██████████| 5370/5370 [00:03<00:00, 1355.98it/s]
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_DIV starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Feature diversity: 0.38793882946575575
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_FPR starts evaluation...
Start evaluation... :  30%|██▉       | 225/751 [00:00<00:00, 571.01it/s]
INFO:cornac.experiment.experiment_explainers:Result: Feature Precision: 0.4161616161616163; Feature Recall: 0.005968494750981334; Harmonic Mean: 0.011440368416677622
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_PSPNFNS st

**Experiment 3**: run FDIV, PGF, MEP and EnDCG on (ALS, Exp_ALS), (EMF, Exp_PHI4MF) and (NEMF, Exp_SU4EMF) pairs using data from movielens. This dataset only contains user, item and rating info.

In [1]:
from cornac.datasets import movielens
from cornac.eval_methods import RatioSplit
from cornac.experiment.experiment_explainers import Experiment_Explainers
from cornac.models import EMF, NEMF, ALS
from cornac.explainer import Exp_ALS, Exp_PHI4MF, Exp_SU4EMF
from cornac.metrics_explainer import (
    Metric_Exp_DIV as DIV,
    Metric_Exp_PGF as PGF,
    Metric_Exp_MEP as MEP,
    Metric_Exp_EnDCG as EnDCG,
)

# Load MovieLens
ml_1m = movielens.load_feedback(variant="100K")

# Define an evaluation method to split feedback into train and test sets
ratio_split = RatioSplit(
    data=ml_1m, test_size=0.2, exclude_unknowns=False, verbose=True
)

# initialize recommenders and explainers
emf = EMF(
    k=10,
    max_iter=500,
    learning_rate=0.001,
    lambda_reg=0.1,
    explain_reg=0.01,
    verbose=True,
    seed=6,
    num_threads=6,
    early_stop=True,
)
nemf = NEMF(
    k=10,
    max_iter=500,
    learning_rate=0.001,
    lambda_reg=0.1,
    explain_reg=0.01,
    novel_reg=1,
    verbose=True,
    seed=6,
    num_threads=6,
    early_stop=True,
)
als = ALS(k=10, max_iter=500, lambda_reg=0.001, alpha=1, verbose=True, seed=6)
als_exp = Exp_ALS(rec_model=als, dataset=ratio_split.train_set)
emf_exp = Exp_PHI4MF(rec_model=emf, dataset=ratio_split.train_set)
nemf_exp = Exp_SU4EMF(rec_model=nemf, dataset=ratio_split.train_set)

# initialize metrics
fdiv = DIV()
pgf = PGF()
mep = MEP()
endcg = EnDCG()

# initialize experiment
models = [(als, als_exp), (emf, emf_exp), (nemf, nemf_exp)]
metrics = [fdiv, pgf, mep, endcg]
experiment = Experiment_Explainers(
    eval_method=ratio_split,
    models=models,
    metrics=metrics,
    distribution=False,
    rec_k=10,
    feature_k=10,
    eval_train=True,
)
experiment.run()

/home/linan/miniconda3/envs/cornac/lib/python3.11/site-packages/tensorflow/python/debug/cli/debugger_cli_common.py:19: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants
INFO:cornac.experiment.experiment_explainers:Start training Recommender ALS...


rating_threshold = 1.0
exclude_unknowns = False
---
Training data:
Number of users = 943
Number of items = 1648
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 943
Number of items = 1682
Number of ratings = 20000
Number of unknown users = 0
Number of unknown items = 34
---
Total users = 943
Total items = 1682


/home/linan/miniconda3/envs/cornac/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 6 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()


  0%|          | 0/500 [00:00<?, ?it/s]

INFO:implicit:Final training loss 0.0616
INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'ALS:Exp_ALS'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender ALS creates recommendations
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_ALS create explanation for all recommendations


...

INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'NEMF:Exp_SU4EMF'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender NEMF creates recommendations


Optimization finished!


INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_SU4EMF create explanation for all recommendations


Computing explanations:   0%|          | 0/9430 [00:00<?, ?it/s]

INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_DIV starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Feature diversity: 0.3279789903784084
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_PGF starts evaluation...
100%|██████████| 943/943 [00:11<00:00, 81.14it/s]
INFO:cornac.experiment.experiment_explainers:Result: Metric_Exp_PGF: 0.1351683980559146
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_MEP starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Metric_Exp_MEP: 0.11930010604453871
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_EnDCG starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Metric_Exp_EnDCG: 0.0640591110133219
INFO:cornac.experiment.experiment_explainers:experiment data: [[0.06432152921362073, 0.2788011509150099, 'N/A', 'N/A', 15.41525936126709, 138.03549456596375], [0.16513618725889898, 0.0, 0.1632025450

**Experiment 4**: Run RA and FA on pairwise models. FA and RA performs comparison between two sets of explainers, thus the score is unavailable when only one explainer is passed.

In [5]:
# prepare data
rs = prepare_data(
    data_name="goodreads",
    test_size=0.2,
    dense=True,
    item=True,
    user=True,
    sample_size=1,
    seed=21,
)

# initialize recommenders and explainers
efm = EFM(
    max_iter=20,
    num_explicit_factors=50,
    num_latent_factors=50,
    num_most_cared_aspects=50,
    alpha=0.85,
    lambda_h=0.1,
    lambda_u=0.001,
    lambda_v=0.01,
    lambda_x=1.0,
    lambda_y=0.1,
)
efm_exp = Exp_EFM(rec_model=efm, dataset=rs.train_set)
mter = MTER(
    max_iter=20,
    n_aspect_factors=8,
    n_item_factors=5,
    n_opinion_factors=5,
    n_user_factors=10,
    lambda_bpr=10,
    lambda_reg=10,
    n_bpr_samples=1000,
    n_element_samples=50,
)
mter_exp = Exp_MTER(rec_model=mter, dataset=rs.train_set)

# initialize metrics
fa = FA()
ra = RA()
fdiv = DIV()

# initialize experiment
models = [[(efm, mter), (efm_exp, mter_exp)], (efm, efm_exp), (mter, mter_exp)]
metrics = [fa, ra, fdiv]
experiment = Experiment_Explainers(
    eval_method=rs,
    models=models,
    metrics=metrics,
    rec_k=10,
    feature_k=10,
    eval_train=True,
)
experiment.run()

INFO:cornac.experiment.experiment_explainers:Start training Recommender1 EFM...
INFO:cornac.experiment.experiment_explainers:Start training Recommender2 MTER...
INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'EFM:Exp_EFM'vs'MTER:Exp_MTER'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Creates fake recommendations from dataset for common used
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer1 Exp_EFM create explanation for all recommendations
Computing explanations: 100%|██████████| 100/100 [00:00<00:00, 6099.83it/s]
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer2 Exp_MTER create explanation for all recommendations
Computing explanations: 100%|██████████| 100/100 [00:00<00:00, 4184.85it/s]
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_FA starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Average Metric_Exp_FA: 0.016666666666666666
INFO:cornac.experiment.expe